In [1]:
import os
# import lxml
from lxml import etree
import pandas as pd
import numpy as np

In [3]:
import sys

In [2]:
from sklearn.metrics import classification_report

In [4]:
import fasttext

In [5]:
import os

from sklearn.model_selection import cross_val_score
import gensim
import pymorphy2
import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn import preprocessing

In [6]:
from sklearn.tree import DecisionTreeClassifier

In [7]:
from sklearn.linear_model import SGDClassifier

In [8]:
from sklearn.preprocessing import Normalizer

In [9]:
from nltk.util import ngrams

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [12]:
cl = ['статья','интервью','комментарий']

In [13]:
f = open('./out.csv', 'r')

file_hash = {}
for line in f:
    attrs = line.split(';')
    path = attrs[0]
    class_name = attrs[8]
    if ' ' in class_name:
        class_name = class_name.split()[0]
    if '|' in class_name:
        class_name = class_name.split('|')[0]
    filename = os.path.basename(path)
    if class_name in cl:
        file_hash[filename] = class_name


In [14]:
morph = pymorphy2.MorphAnalyzer()
def normalize_and_add_pos(word):
    p = morph.parse(word)[0]
    return '{}_{}'.format(
        p.normal_form,
        p.tag.POS
    )

In [15]:
def read_txt(path):
    for root, dirs, files in os.walk(path):
        for filename in files:
            open_name = os.path.join(root, filename)
            if filename in file_hash:
                class_text = file_hash[filename]
                tree = etree.parse(open_name)
                yield tree, class_text

In [16]:
sent = []
for tree, class_text in read_txt('/Users/Valeriya/Desktop/CoursePaper/corpus'):
    all_p = tree.xpath('//p')
    joined = '\n'.join([p.text for p in all_p if p.text is not None])
    sent.append((joined, class_text))

In [17]:
sent = sent[:800]

In [18]:
only_texts = [text[0] for text in sent]
classes = [text[1] for text in sent]

In [19]:
df = pd.DataFrame(only_texts)

In [20]:
df['class'] = classes

In [47]:
X_train, X_test, y_train, y_test = train_test_split(df.ix[:,0], df['class'], test_size=0.3)

In [48]:
count_vect = CountVectorizer()
X_train = count_vect.fit_transform(X_train)
tf_transformer = TfidfTransformer()
X_train = tf_transformer.fit_transform(X_train)

In [49]:
X_test = count_vect.transform(X_test)
X_test = tf_transformer.transform(X_test)

In [50]:
clf_grad = linear_model.SGDClassifier()
clf_grad.fit(X_train, y_train)
predicted = clf_grad.predict(X_test)
print(classification_report(y_test, predicted))

             precision    recall  f1-score   support

   интервью       0.62      0.27      0.38        55
комментарий       0.70      0.68      0.69        76
     статья       0.70      0.92      0.80       109

avg / total       0.69      0.70      0.67       240



In [51]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

   интервью       0.33      0.11      0.16        55
комментарий       0.66      0.76      0.71        76
     статья       0.63      0.78      0.70       109

avg / total       0.57      0.62      0.58       240



In [52]:
svc = SVC()
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)
print(classification_report(y_test, y_pred_svc))

             precision    recall  f1-score   support

   интервью       0.00      0.00      0.00        55
комментарий       0.00      0.00      0.00        76
     статья       0.45      1.00      0.62       109

avg / total       0.21      0.45      0.28       240



/Users/Valeriya/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [53]:
regr = LogisticRegression()
regr.fit(X_train, y_train)
y_pred_regr = regr.predict(X_test)
print(classification_report(y_test, y_pred_regr))

             precision    recall  f1-score   support

   интервью       0.00      0.00      0.00        55
комментарий       0.80      0.54      0.65        76
     статья       0.57      0.98      0.72       109

avg / total       0.51      0.62      0.53       240



In [54]:
y_pred_rest = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train).predict(X_test)
print(classification_report(y_test, y_pred_rest))

             precision    recall  f1-score   support

   интервью       0.59      0.18      0.28        55
комментарий       0.76      0.71      0.73        76
     статья       0.68      0.94      0.79       109

avg / total       0.68      0.70      0.65       240



In [55]:
clf_grad = linear_model.SGDClassifier()
clf_grad.fit(X_train, y_train)
y_pred_grad = clf_grad.predict(X_test)
print(classification_report(y_test, y_pred_grad))

             precision    recall  f1-score   support

   интервью       0.54      0.24      0.33        55
комментарий       0.74      0.70      0.72        76
     статья       0.70      0.93      0.80       109

avg / total       0.68      0.70      0.66       240



In [57]:
min_max_scaler = preprocessing.MaxAbsScaler()
X_train_norm = min_max_scaler.fit_transform(X_train)
X_test_norm = min_max_scaler.fit_transform(X_test)

clf_bayes = MultinomialNB()
clf_bayes.fit(X_train_norm, y_train)
y_pred_bayes = clf_bayes.predict(X_test_norm)

print(classification_report(y_test, y_pred_bayes))

             precision    recall  f1-score   support

   интервью       0.42      0.15      0.22        55
комментарий       0.77      0.43      0.55        76
     статья       0.57      0.93      0.70       109

avg / total       0.60      0.59      0.54       240



In [58]:
clf_tree = DecisionTreeClassifier(random_state=0)
clf_tree.fit(X_train, y_train)
y_pred_tree = clf_tree.predict(X_test)
print(classification_report(y_test, y_pred_tree))

             precision    recall  f1-score   support

   интервью       0.43      0.47      0.45        55
комментарий       0.57      0.64      0.60        76
     статья       0.69      0.59      0.63       109

avg / total       0.59      0.58      0.58       240

